# M1 data double split

In [31]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re

In [32]:
xls = pd.ExcelFile('Master Data Sheet Organization.xlsx')
data_df = pd.read_excel(xls, 'Master Sheet - M1')
new_header = data_df.iloc[0] #grab the first row for the header
data_df = data_df[1:] #take the data less the header row
data_df.columns = new_header #set the header row as the df header
data_df = data_df.loc[(data_df['Picture?'] == 'Y')].reset_index(drop=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,NaN,Country,District,Sentinel Site,Date,MOSQUITO ID (Team),Morph ID Genus,Morph ID Species,PCR Number (Country Specific),PCR ID Genus,PCR ID Species,Sibling Species,Picture?,Prototype,Abdomen Status,Sex,Parity,Eliza Results
0,6.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0006,Anopheles,funestus,498,NaN,funestus,NaN,Y,1409,SG,F,NaN,NaN
1,8.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0008,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
2,9.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0009,Anopheles,funestus,NaN,NaN,funestus,NaN,Y,NaN,G,F,NaN,NEG
3,10.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0010,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,G,F,NaN,NEG
4,11.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0011,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,NaN,NaN,NaN,NEG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1646.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0295,Anopheles,gambiae,MI2100295,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1405,1647.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0296,Anopheles,rufipes,MI2100296,Anopheles,NaN,arabiensis,Y,NaN,UF,F,NaN,NaN
1406,1648.0,Zambia,Matcha,Chidakwa,2021-05-10 00:00:00,Z-MA-01-0297,Anopheles,rufipes,MI2100297,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1407,1649.0,Zambia,Matcha,Chidakwa,2021-05-09 00:00:00,Z-MA-01-0298,Anopheles,longipalps,MI2100298,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN


In [33]:
df_a = data_df[(data_df['Morph ID Genus'] == 'Anopheles') | (data_df['Morph ID Genus'] == 'anopheles') | (data_df['Morph ID Genus'] == 'Anopheles ')]
print(df_a.shape)
df_b = data_df[(data_df['Morph ID Genus'] == 'Culex')]
print(df_b.shape)
df_c = data_df[(data_df['Morph ID Genus'] != 'Culex') & (data_df['Morph ID Genus'] != 'Anopheles') & (data_df['Morph ID Genus'] != 'anopheles')& (data_df['Morph ID Genus'] != 'Anopheles ')]
print(df_c.shape)

(1000, 18)
(224, 18)
(185, 18)


In [34]:
# use PCR results if avaliable
df = pd.DataFrame()
ID = []
genus = []
species = []
for i in range(data_df.shape[0]):
    ID.append(data_df.loc[i,'MOSQUITO ID (Team)'])
    genus_name = data_df.loc[i,'Morph ID Genus']
    species_name = data_df.loc[i,'Morph ID Species']
    if genus_name == 'Anopheles' or genus_name == 'anopheles' or genus_name == 'Anopheles ':
        if pd.notna(data_df.loc[i,'PCR ID Genus']) == True:
            genus_name = data_df.loc[i,'PCR ID Genus']
        if pd.notna(data_df.loc[i,'PCR ID Species']) == True:
            species_name = data_df.loc[i,'PCR ID Species']
    genus.append(genus_name)
    species.append(species_name)
df['ID'] = ID
df['genus'] = genus
df['species'] = species
df = df.dropna(subset=['genus'],how='any').reset_index(drop=True)
df

,ID,genus,species
0,U-SR-01-0006,Anopheles,funestus
1,U-SR-01-0008,Anopheles,funestus
2,U-SR-01-0009,Anopheles,funestus
3,U-SR-01-0010,Anopheles,funestus
4,U-SR-01-0011,Anopheles,funestus
...,...,...,...
1350,Z-MA-01-0295,Anopheles,gambiae
1351,Z-MA-01-0296,Anopheles,rufipes
1352,Z-MA-01-0297,Anopheles,rufipes
1353,Z-MA-01-0298,Anopheles,longipalps


In [35]:
print(len(df[df['genus'] == 'Mansonia']))

20


In [36]:
df_with_label = pd.DataFrame()
label = []
for i in range(df.shape[0]):
    genus = df.loc[i,'genus']
    species = df.loc[i,'species']
    if genus == 'Anopheles' or genus == 'anopheles' or genus == 'Anopheles ':
        if species == 'funestus':
            label.append(0)
        elif species == 'gambiae' or species == 'Gambiae' or species == 'gambie':
            label.append(1)
        else:
            label.append(2)
    elif genus == "Culex" or genus == "Culex ":
        label.append(3)
    else:
        label.append(4)
        
df_with_label['ID'] = df['ID']
df_with_label['label'] = label
df_with_label

,ID,label
0,U-SR-01-0006,0
1,U-SR-01-0008,0
2,U-SR-01-0009,0
3,U-SR-01-0010,0
4,U-SR-01-0011,0
...,...,...
1350,Z-MA-01-0295,1
1351,Z-MA-01-0296,2
1352,Z-MA-01-0297,2
1353,Z-MA-01-0298,2


In [37]:
df1 = df_with_label[df_with_label['label'] == 0]
df2 = df_with_label[df_with_label['label'] == 1]
df3 = df_with_label[df_with_label['label'] == 2]
df4 = df_with_label[df_with_label['label'] == 3]
df5 = df_with_label[df_with_label['label'] == 4]
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(356, 2)
(293, 2)
(351, 2)
(332, 2)
(23, 2)


In [38]:
# one train and test split
# from sklearn.model_selection import train_test_split

# train_data1, test_data1 = train_test_split(df1, test_size=0.15)
# train_data2, test_data2 = train_test_split(df2, test_size=0.15)
# train_data3, test_data3 = train_test_split(df3, test_size=0.15)
# train_data4, test_data4 = train_test_split(df4, test_size=0.15)

In [39]:
train_data = pd.concat(
    [df1,df2,df3,df4, df5], ignore_index=True
)

print(train_data.shape)

(1355, 2)


In [40]:
train_data

,ID,label
0,U-SR-01-0006,0
1,U-SR-01-0008,0
2,U-SR-01-0009,0
3,U-SR-01-0010,0
4,U-SR-01-0011,0
...,...,...
1350,U-SR-01-0712,4
1351,U-SR-01-0716,4
1352,G-TM-05-0001,4
1353,G-TM-05-0002,4


In [41]:
from torchvision import transforms
mosquito_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize([300,300]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [42]:
from PIL import Image, ImageOps
def make_data(d):
    data_path = "/home/shrutihegde/Documents/Shruti/Species/Data/cropped/crop_m1_pad/"
    data = []
    label = []
    paths = []
    F_count = 0
    G_count = 0
    Aother_count = 0
    C_count = 0
    Other_count = 0
    df = pd.DataFrame()
    for i in range(d.shape[0]):
        for f in os.listdir(data_path):
            if re.match(d.loc[i,'ID'], f) and f.endswith('a.jpg'):
                path = os.path.join(data_path, f)
                paths.append(path)
                image = Image.open(path)
                #image = ImageOps.exif_transpose(image)
                image = mosquito_transforms(image)
                data.append(image)
                if d.loc[i,'label'] == 0:
                    label.append(0)
                    F_count += 1
                elif d.loc[i,'label'] == 1:
                    label.append(1)
                    G_count += 1
                elif d.loc[i,'label'] == 2:
                    label.append(2)
                    Aother_count += 1    
                elif d.loc[i,'label'] == 3:
                    label.append(3)
                    C_count += 1
                elif d.loc[i,'label'] == 4:
                    label.append(4)
                    Other_count += 1
    stacked_data = torch.stack(data)
    df['path'] = paths
    df['label'] = label                   
                
    return stacked_data, label, df, F_count, G_count, Aother_count, C_count, Other_count 

In [43]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
skf = StratifiedKFold(n_splits=5, shuffle = True)
target = train_data.loc[:,'label']

target_list=target.values.tolist()

count = 1
for train_index, test_index in skf.split(train_data, target):
    train_val_target_list=[]
    for i in train_index:
        train_val_target_list.append(target[i])
 
    train = train_data.loc[train_index,:]
    test = train_data.loc[test_index,:]
    
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    # one train and test split

    train_data_split, val_data_split = train_test_split(train, test_size=0.20,stratify=train_val_target_list)
    print(train_data_split["ID"])
    
    train_data_split = train_data_split.reset_index(drop=True)
    val_data_split = val_data_split.reset_index(drop=True)
    
    print("Validation: ")
    d, label, path_df, _,_,_,_,_= make_data(val_data_split)
    torch.save(d,f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/val_data_fold{count}.pt")
    torch.save(torch.tensor(label),f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/val_label_fold{count}.pt")
    path_df.to_csv(f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/val_datapath_fold{count}.csv", index = False)
    
    print("Train: ")
    d2, label2, path_df2, _,_,_,_,_ = make_data(train_data_split)
    torch.save(d2,f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/train_data_fold{count}.pt")
    torch.save(torch.tensor(label2),f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/train_label_fold{count}.pt")
    path_df2.to_csv(f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/train_datapath_fold{count}.csv", index = False)
    
    print("Test: ")
    d, label, path_df, _,_,_,_,_ = make_data(test)
    torch.save(d,f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/test_data_fold{count}.pt")
    torch.save(torch.tensor(label),f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/test_label_fold{count}.pt")
    path_df.to_csv(f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M1/test_datapath_fold{count}.csv", index = False)
    
    count += 1

914     G-TM-03-0045
1010    G-TM-04-0056
827     G-TM-02-0029
738     Z-MA-01-0206
180     U-SR-01-0308
            ...     
744     Z-MA-01-0214
444     G-TM-05-0066
907     G-TM-03-0037
25      U-SR-01-0065
870     G-TM-02-0094
Name: ID, Length: 867, dtype: object
Validation: 
Train: 
Test: 
673     Z-MA-01-0065
453     Z-MA-01-0013
772     Z-MA-01-0250
117     U-SR-01-0191
250     U-SR-01-0502
            ...     
795     Z-MA-01-0289
183     U-SR-01-0308
1032    G-TM-04-0078
44      U-SR-01-0088
965     G-TM-03-0098
Name: ID, Length: 867, dtype: object
Validation: 
Train: 
Test: 
988    G-TM-04-0029
13     U-SR-01-0021
797    Z-MA-01-0296
886    G-TM-03-0006
572    G-TM-01-0091
           ...     
296    U-SR-01-0519
880    G-TM-02-0100
583    G-TM-03-0113
388    G-TM-01-0038
358    U-SR-02-0032
Name: ID, Length: 867, dtype: object
Validation: 
Train: 
Test: 
610     G-TM-04-0126
1079    U-SR-01-0711
559     G-TM-01-0072
621     G-TM-04-0141
883     G-TM-02-0095
            ...   

# M2

In [44]:
xls = pd.ExcelFile('Master Data Sheet Organization.xlsx')
data_df = pd.read_excel(xls, 'Master Sheet - M2')
new_header = data_df.iloc[0] #grab the first row for the header
data_df = data_df[1:] #take the data less the header row
data_df.columns = new_header #set the header row as the df header
data_df.reset_index(drop=True, inplace=True)
data_df

,mosquito_id,Country,City,Site,Date,specimen_image_1,specimen_image_2,specimen_image_3,morph_id_genus,morph_id_genus_other,...,pcr_present,pcr_id_genus,pcr_id_genus_other,pcr_id_aedes_species,pcr_id_anopheles_species,pcr_id_species_other,pcr_id_sibling,additional_note,PARENT_KEY,KEY
0,G031B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900605929.jpg,1648900637908.jpg,1648900654087.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
1,G032B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900707180.jpg,1648900718477.jpg,1648900736152.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
2,G033B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900875092.jpg,1648900888835.jpg,1648900905444.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
3,G034B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900946763.jpg,1648900961718.jpg,1648900978547.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
4,B035B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648901102036.jpg,1648901120450.jpg,1648901136984.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,NaN,NaN,NaN,NaN,NaN,1643193598508.jpg,1643193634018.jpg,1643193701849.jpg,Anopheles,\n,...,later,\n,\n,\n,\n,\n,\n,Na,uuid:b8df0317-2b65-4d17-afe9-fd7ed0f52609,uuid:b8df0317-2b65-4d17-afe9-fd7ed0f52609/hh_r...
710,NaN,NaN,NaN,NaN,NaN,1643192970690.jpg,1643193019263.jpg,1643193065582.jpg,Anopheles,\n,...,later,\n,\n,\n,\n,\n,\n,NaN,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3/hh_r...
711,NaN,NaN,NaN,NaN,NaN,1643193284792.jpg,1643193320535.jpg,1643193372510.jpg,Anopheles,\n,...,later,\n,\n,\n,\n,\n,\n,Na,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3/hh_r...
712,NaN,NaN,NaN,NaN,NaN,1642774690032.jpg,1642774707378.jpg,1642774718884.jpg,Anopheles,\n,...,no,\n,\n,\n,\n,\n,\n,NaN,uuid:8b60ca61-4660-4ff6-af4b-7e1cfd79b967,uuid:8b60ca61-4660-4ff6-af4b-7e1cfd79b967/hh_r...


In [45]:
data_df.reset_index(drop=True, inplace=True)
data_df.shape

(714, 28)

In [46]:
label = []
for i in range(data_df.shape[0]):
    genus = data_df.loc[i,'morph_id_genus']
    species = data_df.loc[i,'morph_id_anopheles_species']
    if genus == 'Anopheles' or genus == 'anopheles' or genus == 'Anopheles ':
        if species == 'funestus':
            label.append(0)
        elif species == 'gambiae' or species == 'Gambiae' or species == 'gambie':
            label.append(1)
        else:
            label.append(2)
    elif genus == "Culex" or genus == "culex ":
        label.append(3)
    else:
        label.append(4)
data_df["label"] = label
data_df

,mosquito_id,Country,City,Site,Date,specimen_image_1,specimen_image_2,specimen_image_3,morph_id_genus,morph_id_genus_other,...,pcr_id_genus,pcr_id_genus_other,pcr_id_aedes_species,pcr_id_anopheles_species,pcr_id_species_other,pcr_id_sibling,additional_note,PARENT_KEY,KEY,label
0,G031B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900605929.jpg,1648900637908.jpg,1648900654087.jpg,Anopheles,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...,1
1,G032B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900707180.jpg,1648900718477.jpg,1648900736152.jpg,Anopheles,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...,1
2,G033B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900875092.jpg,1648900888835.jpg,1648900905444.jpg,Anopheles,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...,1
3,G034B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900946763.jpg,1648900961718.jpg,1648900978547.jpg,Anopheles,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...,1
4,B035B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648901102036.jpg,1648901120450.jpg,1648901136984.jpg,Anopheles,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,NaN,NaN,NaN,NaN,NaN,1643193598508.jpg,1643193634018.jpg,1643193701849.jpg,Anopheles,\n,...,\n,\n,\n,\n,\n,\n,Na,uuid:b8df0317-2b65-4d17-afe9-fd7ed0f52609,uuid:b8df0317-2b65-4d17-afe9-fd7ed0f52609/hh_r...,1
710,NaN,NaN,NaN,NaN,NaN,1643192970690.jpg,1643193019263.jpg,1643193065582.jpg,Anopheles,\n,...,\n,\n,\n,\n,\n,\n,NaN,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3/hh_r...,1
711,NaN,NaN,NaN,NaN,NaN,1643193284792.jpg,1643193320535.jpg,1643193372510.jpg,Anopheles,\n,...,\n,\n,\n,\n,\n,\n,Na,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3/hh_r...,1
712,NaN,NaN,NaN,NaN,NaN,1642774690032.jpg,1642774707378.jpg,1642774718884.jpg,Anopheles,\n,...,\n,\n,\n,\n,\n,\n,NaN,uuid:8b60ca61-4660-4ff6-af4b-7e1cfd79b967,uuid:8b60ca61-4660-4ff6-af4b-7e1cfd79b967/hh_r...,0


In [47]:
# species
df1 = data_df[data_df['label'] == 0]
df2 = data_df[data_df['label'] == 1]
df3 = data_df[data_df['label'] == 2]
df4 = data_df[data_df['label'] == 3]
df5 = data_df[data_df['label'] == 4]

# # sex
# df1 = data_df[data_df['specimen_gender'] == 'F']
# df2 = data_df[data_df['specimen_gender'] == 'M']

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(238, 29)
(223, 29)
(37, 29)
(213, 29)
(3, 29)


In [48]:
df1 = df1.sample(frac=1).reset_index(drop=True)
df2 = df2.sample(frac=1).reset_index(drop=True)
df3 = df3.sample(frac=1).reset_index(drop=True)
df4 = df4.sample(frac=1).reset_index(drop=True)
df5 = df5.sample(frac=1).reset_index(drop=True)
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(238, 29)
(223, 29)
(37, 29)
(213, 29)
(3, 29)


In [49]:
train_data = pd.concat(
    [df1,df2,df3,df4, df5], ignore_index=True
)

print(train_data.shape)

(714, 29)


In [81]:
train_data['label']

0      0
1      0
2      0
3      0
4      0
      ..
709    3
710    3
711    4
712    4
713    4
Name: label, Length: 714, dtype: int64

In [50]:
from torchvision import transforms
mosquito_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize([300,300]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [66]:
from PIL import Image, ImageOps
def make_data_m2(d):
    data_path = "/home/shrutihegde/Documents/Shruti/Species/Data/cropped/crop_m2_pad"
    data = []
    label = []
    paths = []
    abdomen_status = []
    sex = []
    F_count = 0
    G_count = 0
    Aother_count = 0
    C_count = 0
    Other_count = 0
    SG_count = 0
    GG_count = 0
    UF_count = 0
    FF_count = 0
    male_count = 0 
    female_count = 0
    df = pd.DataFrame()
    for i in range(d.shape[0]):
        list_specimen = []
        list_specimen.append(d.loc[i,'specimen_image_1'])
        list_specimen.append(d.loc[i,'specimen_image_2'])
        list_specimen.append(d.loc[i,'specimen_image_3'])
        for specimen in list_specimen:
            if specimen in os.listdir(data_path):
                path = os.path.join(data_path, specimen)
                paths.append(path)
                image = Image.open(path)
                #image = ImageOps.exif_transpose(image)
                image = mosquito_transforms(image)
                data.append(image)
                if d.loc[i,'label'] == 0:
                    label.append(0)
                    F_count += 1
                elif d.loc[i,'label'] == 1:
                    label.append(1)
                    G_count += 1
                elif d.loc[i,'label'] == 2:
                    label.append(2)
                    Aother_count += 1    
                elif d.loc[i,'label'] == 3:
                    label.append(3)
                    C_count += 1
                elif d.loc[i,'label'] == 4:
                    label.append(4)
                    Other_count += 1
                if d.loc[i,'specimen_characteristics'] == "SG":
                    abdomen_status.append(0)
                    SG_count += 1
                if d.loc[i,'specimen_characteristics'] == "G":
                    abdomen_status.append(1)
                    GG_count += 1
                if d.loc[i,'specimen_characteristics'] == "UF":
                    abdomen_status.append(2)
                    UF_count += 1
                if d.loc[i,'specimen_characteristics'] == "FF":
                    abdomen_status.append(3)
                    FF_count += 1
                if d.loc[i,'specimen_gender'] == "F":
                    sex.append(0)
                    female_count += 1
                if d.loc[i,'specimen_gender'] == "M":
                    sex.append(1)
                    male_count += 1
                    
    stacked_data = torch.stack(data)
    df['path'] = paths
    df['label'] = label
    df['abdomen'] = abdomen_status
    df['gender'] = sex 
    print("F_count: ", F_count)
    print("G_count: ", G_count)
    print("Aother_count: ", Aother_count)
    print("C_count: ", C_count)
    print("Other_count: ", Other_count)
    
    d1={'label':label, 'abdomen':abdomen_status, 'gender':sex}
                
    return stacked_data, d1, df

In [78]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
skf = StratifiedKFold(n_splits=5, shuffle = True)
target = train_data.loc[:,'label']

target_list=target.values.tolist()

count = 1
for train_index, test_index in skf.split(train_data, target):
    train_val_target_list=[]
    for i in train_index:
        train_val_target_list.append(target[i])
 
    train = train_data.loc[train_index,:]
    test = train_data.loc[test_index,:]
    
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    # one train and test split

    train_data_split, val_data_split = train_test_split(train, test_size=0.20,stratify=train_val_target_list)
    print(train_data_split["mosquito_id"])
    
    train_data_split = train_data_split.reset_index(drop=True)
    val_data_split = val_data_split.reset_index(drop=True)
    
    print("Validation: ")
    d1, label1, path_df1= make_data_m2(val_data_split)
    torch.save(d,f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/val_data_fold{count}.pt")
    torch.save((label1),f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/val_label_fold{count}.pt")
    path_df.to_csv(f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/val_datapath_fold{count}.csv", index = False)
    
    print("Train: ")
    d2, label2, path_df2 = make_data_m2(train_data_split)
    torch.save(d2,f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/train_data_fold{count}.pt")
    torch.save((label2),f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/train_label_fold{count}.pt")
    path_df2.to_csv(f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/train_datapath_fold{count}.csv", index = False)
    
    print("Test: ")
    d3, label3, path_df3 = make_data_m2(test)
    torch.save(d,f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/test_data_fold{count}.pt")
    torch.save((label3),f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/test_label_fold{count}.pt")
    path_df.to_csv(f"/home/shrutihegde/Documents/Shruti/Species/Data/CV_1_M2/test_datapath_fold{count}.csv", index = False)
    
    count += 1

/home/shrutihegde/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


221    G256B
61     F198M
351    G504M
120    F029m
47     F216M
       ...  
38     F217M
297      NaN
359    Goo5B
509    C496m
554    Bc-72
Name: mosquito_id, Length: 456, dtype: object
Validation: 
F_count:  87
G_count:  93
Aother_count:  18
C_count:  75
Other_count:  0
Train: 
F_count:  359
G_count:  369
Aother_count:  54
C_count:  339
Other_count:  3
Test: 
F_count:  114
G_count:  108
Aother_count:  15
C_count:  96
Other_count:  0
429    C218B
357    G110B
78      F399
39     F110m
563    Bc-77
       ...  
272    G065B
9      F106m
513      C09
208    G012B
391      NaN
Name: mosquito_id, Length: 456, dtype: object
Validation: 
F_count:  96
G_count:  102
Aother_count:  15
C_count:  78
Other_count:  0
Train: 
F_count:  348
G_count:  354
Aother_count:  57
C_count:  333
Other_count:  0
Test: 
F_count:  116
G_count:  114
Aother_count:  15
C_count:  99
Other_count:  3
184    F250M
429    C230B
0      F121m
124    F203m
98     F078m
       ...  
368    G033B
162    F001m
112    F042M
